> Add the tailored models : 

In [ ]:
import os 

path_work = "/media/concha-eloko/Linux/PPT_clean/ficheros_28032023"
path_server = f"{path_work}/ensemble_tailored"
path_old_ensemble = f"{path_work}/ensemble_2809"
path_ensemble = f"{path_work}/ensemble_tailored_0612"

tailored_KLtypes = ["KL3" , "KL8", "KL12", "KL13", "KL23", "KL38", "KL46" ,"KL62"]
# Make sure the rep is created :
try :
    os.mkdir(path_ensemble)
except FileExistsError :
    pass

# Get the regulars there :
for model in os.listdir(path_old_ensemble) : 
    kltype = model.split(".")[0]
    if kltype not in tailored_KLtypes :
        os.system(f"cp {path_old_ensemble}/{model} {path_ensemble}")

# Get the tailored ones :
for model in os.listdir(path_server) : 
    kltype = model.split(".")[0]
    if kltype in tailored_KLtypes :
        os.system(f"cp {path_server}/{model} {path_ensemble}")

***
# Prediction code : 

In [1]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

import TropiGAT_functions 
#from TropiGAT_functions import get_top_n_kltypes ,clean_print 

import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool
warnings.filterwarnings("ignore")

# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
path_ensemble = f"{path_work}/ficheros_28032023/ensemble_tailored_0612"



> Make model : 

In [2]:
dico_tailored = {"KL3" : "small", "KL62" : "small"}
dico_models = TropiGAT_functions.make_ensemble_tailored_TropiGAT(path_ensemble , dico_tailored)

> Ferriol

In [5]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_Dpo_domain_org = "/media/concha-eloko/Linux/depolymerase_building/clean_77_phages_depo"

dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.csv", sep = "," , header = None)
dpo_embeddings = dpo_embeddings.drop([1281] , axis = 1)
dpo_embeddings.set_index([0], inplace = True)

In [10]:
sorted(dpo_embeddings.index.tolist())

['K10PH82C1__cds_50_Dpo_domain',
 'K10PH82C1__cds_51_Dpo_domain',
 'K11PH164C1__cds_45_Dpo_domain',
 'K11PH164C1__cds_46_Dpo_domain',
 'K13PH07C1S__cds_10_Dpo_domain',
 'K13PH07C1S__cds_11_Dpo_domain',
 'K14PH164C1__cds_24_Dpo_domain',
 'K15PH90__cds_55_Dpo_domain',
 'K16PH164C3__cds_48_Dpo_domain',
 'K17alfa61__cds_23_Dpo_domain',
 'K17alfa62__cds_64_Dpo_domain',
 'K1PH164C1__cds_8_Dpo_domain',
 'K21lambda1__cds_28_Dpo_domain',
 'K22PH164C1__cds_10_Dpo_domain',
 'K23PH08C2__cds_233_Dpo_domain',
 'K24PH164C1__cds_8_Dpo_domain',
 'K25PH129C1__cds_60_Dpo_domain',
 'K26PH128C1__cds_49_Dpo_domain',
 'K26PH128C1__cds_50_Dpo_domain',
 'K27PH129C1__cds_48_Dpo_domain',
 'K2PH164C1__cds_23_Dpo_domain',
 'K2PH164C2__cds_24_Dpo_domain',
 'K2alfa62__cds_23_Dpo_domain',
 'K35PH164C3__cds_48_Dpo_domain',
 'K37PH164C1__cds_47_Dpo_domain',
 'K37PH164C1__cds_48_Dpo_domain',
 'K38PH09C2__cds_24_Dpo_domain',
 'K39PH122C2__cds_55_Dpo_domain',
 'K39PH122C2__cds_8_Dpo_domain',
 'K40PH129C1__cds_56_Dpo_domai

> Bea

In [4]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

bea_embeddings = pd.read_csv(f"{path_project}/Bea_phages.esm2.embedding.csv", sep = "," , header = None)
bea_embeddings = bea_embeddings.drop([1281] , axis = 1)
bea_embeddings.set_index([0], inplace = True)


> Towndsend

In [5]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

towndsend_embeddings = pd.read_csv(f"{path_project}/Townsed_phages.esm2.embedding.csv", sep = "," , header = None)
towndsend_embeddings = towndsend_embeddings.drop([1281] , axis = 1)
towndsend_embeddings.set_index([0], inplace = True)


> Others 

In [6]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

others_embeddings = pd.read_csv(f"{path_project}/Others_all.esm2.embedding.csv", sep = "," , header = None)
others_embeddings.set_index([0], inplace = True)

***
> run the predictions 

In [7]:
# Run the predictions ferriol :
ferriol_predictions = {}
for dpo in dpo_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([dpo_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    ferriol_predictions[dpo] = pred

In [9]:
ferriol_predictions

{'K15PH90__cds_55_Dpo_domain': {'KL145': 0.7034,
  'KL8': 0.6928,
  'KL48': 0.9215,
  'KL7': 0.9829,
  'KL118': 0.9609,
  'KL15': 0.9745,
  'KL12': 0.986,
  'KL43': 0.8158,
  'KL136': 0.9763,
  'KL13': 0.9493,
  'KL18': 0.6678,
  'KL149': 0.5068},
 'K80PH1317b__cds_54_Dpo_domain': {'KL128': 0.756,
  'KL74': 0.9412,
  'KL9': 0.6034,
  'KL48': 0.5076,
  'KL81': 0.8484,
  'KL110': 0.5457,
  'KL149': 0.5951},
 'K64PH164C4__cds_24_Dpo_domain': {'KL145': 0.5135,
  'KL14': 0.8466,
  'KL64': 0.9529,
  'KL123': 0.8029,
  'KL110': 0.722,
  'KL21': 0.7637},
 'K5lambda5__cds_196_Dpo_domain': {'KL145': 0.9503,
  'KL74': 0.817,
  'KL48': 0.8594,
  'KL52': 0.5992,
  'KL45': 0.5474,
  'KL118': 0.5642,
  'KL2': 0.6205,
  'KL53': 0.8357,
  'KL127': 0.7403,
  'KL136': 0.8242,
  'KL110': 0.6704,
  'KL18': 0.976,
  'KL149': 0.6745},
 'K11PH164C1__cds_46_Dpo_domain': {'KL145': 0.9378,
  'KL74': 0.9436,
  'KL8': 0.5598,
  'KL45': 0.6045,
  'KL57': 0.5124,
  'KL70': 0.8304,
  'KL127': 0.9122,
  'KL81': 0.5269

In [8]:
# format the results : 
ferriol_pred_formated = TropiGAT_functions.format_predictions(ferriol_predictions , sep = "__")
TropiGAT_functions.clean_print(ferriol_pred_formated)

{'K10PH82C1': {'KL10': 0.5707,
               'KL110': 0.9517,
               'KL118': 0.5704,
               'KL12': 0.9335,
               'KL127': 0.8499,
               'KL128': 0.9694,
               'KL136': 0.7698,
               'KL2': 0.6289,
               'KL25': 0.6036,
               'KL34': 0.9265,
               'KL45': 0.7,
               'KL48': 0.8131,
               'KL53': 0.9169,
               'KL60': 0.9817,
               'KL62': 0.8534,
               'KL63': 0.5017,
               'KL74': 0.796},
 'K11PH164C1': {'KL105': 0.5814,
                'KL111': 0.7759,
                'KL127': 0.9122,
                'KL145': 0.9378,
                'KL24': 0.5383,
                'KL39': 0.5719,
                'KL45': 0.6045,
                'KL57': 0.5124,
                'KL60': 0.8816,
                'KL64': 0.5531,
                'KL70': 0.8304,
                'KL74': 0.9436,
                'KL8': 0.5598,
                'KL81': 0.7456,
                'KL9'

***

In [10]:
# Run the predictions Bea :
bea_predictions = {}
for dpo in bea_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([bea_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    bea_predictions[dpo] = pred

In [11]:
# format the results : 
bea_pred_formated = TropiGAT_functions.format_predictions(bea_predictions , sep = "_")
TropiGAT_functions.clean_print(bea_pred_formated)

{'A1a': {'KL110': 0.9635,
         'KL123': 0.9964,
         'KL125': 0.7587,
         'KL127': 0.6217,
         'KL14': 0.5576,
         'KL145': 0.6561,
         'KL18': 0.5223,
         'KL2': 0.5835,
         'KL28': 0.5783,
         'KL29': 0.8226,
         'KL39': 0.5847,
         'KL57': 0.7262,
         'KL7': 0.9813,
         'KL70': 0.9127,
         'KL74': 0.9737,
         'KL9': 0.9738},
 'A1b': {'KL1': 0.646,
         'KL110': 0.9806,
         'KL114': 0.6447,
         'KL122': 0.7035,
         'KL123': 0.9989,
         'KL125': 0.9129,
         'KL127': 0.5555,
         'KL128': 0.8204,
         'KL136': 0.5726,
         'KL14': 0.6044,
         'KL145': 0.8987,
         'KL18': 0.64,
         'KL28': 0.5022,
         'KL29': 0.5146,
         'KL34': 0.8648,
         'KL39': 0.5375,
         'KL43': 0.6345,
         'KL48': 0.6458,
         'KL5': 0.597,
         'KL53': 0.9885,
         'KL57': 0.6657,
         'KL60': 0.9888,
         'KL63': 0.563,
         'KL7': 0.99

In [ ]:
towndsend_embeddings.info()

In [12]:
# Run the predictions Towndsend :
towndsend_predictions = {}
for dpo in towndsend_embeddings.index : 
    graph_dpo = TropiGAT_functions.make_query_graph([towndsend_embeddings.loc[dpo].values])
    pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
    towndsend_predictions[dpo] = pred

In [13]:
# format the results : 
towndsend_pred_formated = TropiGAT_functions.format_predictions(towndsend_predictions , sep = "_")
TropiGAT_functions.clean_print(towndsend_pred_formated)

{'BLCJPOBP': {'KL110': 0.9665,
              'KL123': 0.9974,
              'KL125': 0.8095,
              'KL127': 0.5507,
              'KL14': 0.6069,
              'KL145': 0.6233,
              'KL57': 0.6884,
              'KL7': 0.9892,
              'KL70': 0.8341,
              'KL74': 0.8679,
              'KL9': 0.9512},
 'DIMCIIMF': {'KL10': 0.9462,
              'KL105': 0.9927,
              'KL106': 0.992,
              'KL108': 0.9986,
              'KL110': 0.9494,
              'KL114': 0.9831,
              'KL116': 0.9865,
              'KL118': 0.9904,
              'KL12': 0.9885,
              'KL123': 0.9984,
              'KL125': 0.9881,
              'KL127': 0.9494,
              'KL128': 0.8972,
              'KL13': 0.9757,
              'KL136': 0.7737,
              'KL14': 0.5173,
              'KL145': 0.863,
              'KL149': 0.9996,
              'KL15': 0.7064,
              'KL16': 1.0,
              'KL17': 0.8348,
              'KL18': 0.845

***

In [ ]:
others_embeddings.info()

In [ ]:
others_embeddings.index

In [14]:
# Run the predictions Others :
other_predictions = {}
for index, dpo in enumerate(others_embeddings.index) :
    if dpo not in ["MN781108.1_prot_QGZ15323.1_262"] :
        try : 
            graph_dpo = TropiGAT_functions.make_query_graph([others_embeddings.loc[dpo].values])
            pred = TropiGAT_functions.run_prediction(graph_dpo,dico_models)
            other_predictions[dpo] = pred
        except Exception as e :
            print(e, dpo)

 ON146449.1_prot_UPW35150.1_13
 ON146449.1_prot_UPW35138.1_1
 ON146449.1_prot_UPW35150.1_13
 ON146449.1_prot_UPW35138.1_1


In [15]:
# format the results : 
others_pred_formated = TropiGAT_functions.format_predictions(other_predictions , sep = "_prot_")
TropiGAT_functions.clean_print(other_predictions)

{'AB716666.1_prot_BAP15736.1_24': {'KL110': 0.9716,
                                   'KL123': 0.9985,
                                   'KL125': 0.8789,
                                   'KL127': 0.5852,
                                   'KL14': 0.7169,
                                   'KL145': 0.8723,
                                   'KL18': 0.578,
                                   'KL28': 0.554,
                                   'KL43': 0.5237,
                                   'KL57': 0.7509,
                                   'KL7': 0.9922,
                                   'KL70': 0.8128,
                                   'KL74': 0.9655,
                                   'KL9': 0.9793},
 'AB716666.1_prot_BAP15746.1_34': {'KL111': 0.5726,
                                   'KL12': 0.6526,
                                   'KL128': 0.9025,
                                   'KL149': 0.698,
                                   'KL15': 0.6228,
                           

In [16]:
predictions = [other_predictions]

with open("/media/concha-eloko/Linux/PPT_clean/TropiGAT.Others.all.tailored.results.tsv", "w") as outfile:
    for prediction in predictions:
        prediction_sorted = dict(sorted(prediction.items()))
        for prot in prediction_sorted:
            if prediction_sorted[prot] == "No hits" or len(prediction_sorted[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction_sorted[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

***
# Write the results : 

In [17]:
predictions = [ferriol_predictions , bea_predictions , towndsend_predictions]

with open("/media/concha-eloko/Linux/PPT_clean/TropiGAT.tailored.results.tsv", "w") as outfile:
    for prediction in predictions:
        for prot in prediction:
            if prediction[prot] == "No hits" or len(prediction[prot]) == 0:
                outfile.write(f"{prot}\tNo hits\n")
            else:
                outfile.write(f"{prot}\t")
                hits = [f"{kltype}:{round(score, 3)}" for kltype, score in prediction[prot].items()]
                sorted_hits = " ; ".join(sorted(hits, key=lambda x: float(x.split(":")[1]), reverse=True))
                outfile.write(sorted_hits)
                outfile.write("\n")

> Parse TropiGAT and Seqbased results :

In [1]:
import pandas as pd 
import os 

path_project = "/media/concha-eloko/Linux/PPT_clean"

tropigat_results = pd.read_csv(f"{path_project}/TropiGAT.tailored.results.tsv", sep = "\t", names = ["protein", "predictions_tropigat"])
# Seqbased_model.results.bit50.tsv
# Seqbased_model.results.tsv
seqbased_results = pd.read_csv(f"{path_project}/Seqbased_model.results.bit50.tsv", sep = "\t", names = ["protein", "predictions_seqbased"])


In [2]:
tropigat_results["protein_id"] = tropigat_results["protein"].apply(lambda x : x.split("_Dpo")[0])
seqbased_results["protein_id"] = seqbased_results["protein"].apply(lambda x : x.split("_A_")[0])

merged_df = tropigat_results.merge(seqbased_results, on='protein_id', how='inner')
merged_df["phage"] = merged_df["protein_id"].apply(lambda x : x.split("__")[0] if x.count("__")>0 else x.split("_")[0])

merged_df_sorted = merged_df.sort_values(by='phage', ascending=True)
merged_df_sorted

,protein_x,predictions_tropigat,protein_id,protein_y,predictions_seqbased,phage
120,A1a_00002,KL123:0.996 ; KL7:0.981 ; KL9:0.974 ; KL110:0....,A1a_00002,A1a_00002,KL7:0.989;KL102:0.569,A1a
105,A1a_00014,KL74:0.974 ; KL70:0.907 ; KL29:0.823 ; KL110:0...,A1a_00014,A1a_00014,KL151:0.708,A1a
106,A1b_00048,KL53:0.989 ; KL60:0.989 ; KL34:0.865 ; KL128:0...,A1b_00048,A1b_00048,KL157:0.57,A1b
98,A1b_00036,KL123:0.999 ; KL7:0.998 ; KL110:0.981 ; KL9:0....,A1b_00036,A1b_00036,KL7:0.989;KL102:0.569,A1b
129,A1c_00046,KL123:0.996 ; KL7:0.992 ; KL110:0.967 ; KL9:0....,A1c_00046,A1c_00046,KL7:0.989;KL102:0.569,A1c
...,...,...,...,...,...,...
128,S13a_00036,KL12:0.979 ; KL70:0.965 ; KL136:0.934 ; KL123:...,S13a_00036,S13a_00036,KL38:0.699,S13a
90,S13b_00058,KL47:0.992 ; KL64:0.913 ; KL28:0.882 ; KL34:0....,S13b_00058,S13b_00058,KL63:0.893,S13b
122,S13c_00055,KL12:0.988 ; KL123:0.976 ; KL145:0.972 ; KL136...,S13c_00055,S13c_00055,KL38:0.9,S13c
121,S13d_00057,KL14:0.99 ; KL21:0.975 ; KL13:0.741 ; KL53:0.5...,S13d_00057,S13d_00057,KL14:0.976,S13d


In [3]:
final_df = merged_df_sorted[["phage","protein_id", "predictions_seqbased", "predictions_tropigat"]]

final_df

,phage,protein_id,predictions_seqbased,predictions_tropigat
120,A1a,A1a_00002,KL7:0.989;KL102:0.569,KL123:0.996 ; KL7:0.981 ; KL9:0.974 ; KL110:0....
105,A1a,A1a_00014,KL151:0.708,KL74:0.974 ; KL70:0.907 ; KL29:0.823 ; KL110:0...
106,A1b,A1b_00048,KL157:0.57,KL53:0.989 ; KL60:0.989 ; KL34:0.865 ; KL128:0...
98,A1b,A1b_00036,KL7:0.989;KL102:0.569,KL123:0.999 ; KL7:0.998 ; KL110:0.981 ; KL9:0....
129,A1c,A1c_00046,KL7:0.989;KL102:0.569,KL123:0.996 ; KL7:0.992 ; KL110:0.967 ; KL9:0....
...,...,...,...,...
128,S13a,S13a_00036,KL38:0.699,KL12:0.979 ; KL70:0.965 ; KL136:0.934 ; KL123:...
90,S13b,S13b_00058,KL63:0.893,KL47:0.992 ; KL64:0.913 ; KL28:0.882 ; KL34:0....
122,S13c,S13c_00055,KL38:0.9,KL12:0.988 ; KL123:0.976 ; KL145:0.972 ; KL136...
121,S13d,S13d_00057,KL14:0.976,KL14:0.99 ; KL21:0.975 ; KL13:0.741 ; KL53:0.5...


In [4]:
final_df.to_csv(f"{path_project}/PPT_results.matrices.tailored_bit50.tsv", sep = "\t", header = True, index = False)